In [12]:
# Imports
import pandas as pd
import os
from sqlalchemy import create_engine

## CSV to DataFrame

In [20]:
# import health_csv 
health14_df = pd.read_csv('../Desktop/Health_Ins_2014.csv', encoding='latin-1')
health15_df = pd.read_csv('../Desktop/Health_Ins_2015.csv')
health16_df = pd.read_csv('../Desktop/Health_Ins_2016.csv')
health17_df = pd.read_csv('../Desktop/Health_Ins_2017.csv')

In [21]:
health14_df.head()

,State,County,Metal Level,Issuer Name,Plan ID (standard component),Plan Marketing Name,Plan Type,Rating Area,Child Only Offering,Source,...,Drug Maximum Out of Pocket - Family - standard,Primary Care Physician - standard,Specialist - standard,Emergency Room - standard,Inpatient Facility - standard,Inpatient Physician - standard,Generic Drugs - standard,Preferred Brand Drugs - standard,Non-preferred Brand Drugs - standard,Specialty Drugs - standard
0,AK,ALEUTIANS EAST,Gold,Premera Blue Cross Blue Shield of Alaska,38344AK0930003,"Blue Cross Blue Shield Select 1000, a Multi-St...",PPO,Rating Area 2,Allows Adult and Child-Only,OPM,...,Included in Medical,$30,$30,20% Coinsurance after deductible,20% Coinsurance after deductible,20% Coinsurance after deductible,$10,$25,$45,20%
1,AK,ALEUTIANS EAST,Gold,Premera Blue Cross Blue Shield of Alaska,38344AK0930004,"Blue Cross Blue Shield Select 1500, a Multi-St...",PPO,Rating Area 2,Allows Adult and Child-Only,OPM,...,Included in Medical,$30,$30,20% Coinsurance after deductible,20% Coinsurance after deductible,20% Coinsurance after deductible,$10,$25,$45,20%
2,AK,ALEUTIANS EAST,Silver,Premera Blue Cross Blue Shield of Alaska,38344AK0930005,"Blue Cross Blue Shield Select 2000, a Multi-St...",PPO,Rating Area 2,Allows Adult and Child-Only,OPM,...,Included in Medical,$35,$35,20% Coinsurance after deductible,20% Coinsurance after deductible,20% Coinsurance after deductible,$15,$50,$100,30% Coinsurance after deductible
3,AK,ALEUTIANS EAST,Silver,Premera Blue Cross Blue Shield of Alaska,38344AK0950001,"Blue Cross Blue Shield Select 1700 HSA, a Mult...",PPO,Rating Area 2,Allows Adult and Child-Only,OPM,...,Included in Medical,20% Coinsurance after deductible,20% Coinsurance after deductible,20% Coinsurance after deductible,20% Coinsurance after deductible,20% Coinsurance after deductible,20% Coinsurance after deductible,20% Coinsurance after deductible,20% Coinsurance after deductible,20% Coinsurance after deductible
4,AK,ALEUTIANS EAST,Bronze,Premera Blue Cross Blue Shield of Alaska,38344AK0950003,"Blue Cross Blue Shield Select 3850 HSA, a Mult...",PPO,Rating Area 2,Allows Adult and Child-Only,OPM,...,Included in Medical,20% Coinsurance after deductible,20% Coinsurance after deductible,20% Coinsurance after deductible,20% Coinsurance after deductible,20% Coinsurance after deductible,20% Coinsurance after deductible,20% Coinsurance after deductible,20% Coinsurance after deductible,20% Coinsurance after deductible


## Transform

In [19]:
# Health Columns wanted for analysis
new_health14_df = pd.DataFrame(health14_df)
health14_standard = new_health14_df[ ['State', 'Plan Marketing Name', 'Plan ID (Standard Component)', 'Metal Level', 'Plan Type',  'Medical Maximum Out Of Pocket - Individual - Standard',
'Medical Maximum Out Of Pocket - Family - Standard', 'Medical Maximum Out Of Pocket - Family (Per Person) - Standard'] ].copy()

health_standard.head()

KeyError: "['Plan ID (Standard Component)', 'Medical Maximum Out Of Pocket - Family (Per Person) - Standard', 'Medical Maximum Out Of Pocket - Family - Standard', 'Medical Maximum Out Of Pocket - Individual - Standard'] not in index"

In [ ]:
# Rename Columns for health and dental standard
transformed_health_standard = health_standard.rename(columns={"Plan ID (Standard Component)": "Plan ID",
                                                              "Medical Maximum Out Of Pocket - Individual - Standard": "Individual Standard",
                                                              "Medical Maximum Out Of Pocket - Family - Standard": "Family Standard",
                                                              "Medical Maximum Out Of Pocket - Family (Per Person) - Standard": "Family Per Person Standard"}).copy()
# drop duplicates of Plan Name (want all different plan types)
transformed_health_standard.drop_duplicates("Plan Marketing Name", inplace=True)
transformed_health_standard.set_index("Plan Type", inplace=True)

transformed_health_standard.head()

In [ ]:
print(transformed_health_standard.count())

### Create Database Connection

In [ ]:
connection_string = "postgres:juiceman@localhost:5432/ETL_Project"
engine = create_engine(f'postgresql://{connection_string}')

###  Load DataFrames into database

In [ ]:
transformed_health_standard.to_sql(name='health', con=engine, if_exists='append', index=True)

In [ ]:
transformed_dental_standard.to_sql(name='dental', con=engine, if_exists='append', index=True)

In [ ]:
 # Confirm tables
engine.table_names()